In [1]:
!pip install faker

In [13]:
import sqlite3
from faker import Faker
import random
from datetime import date

# Initialize Faker
fake = Faker()

# Connect to SQLite database (or create it)
conn = sqlite3.connect('courier_services13.db')
cursor = conn.cursor()

# Create tables for the courier services database
def create_tables():
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Packages (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        weight REAL NOT NULL,
        length REAL NOT NULL,
        width REAL NOT NULL,
        height REAL NOT NULL,
        content_description TEXT NOT NULL
    )''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Receivers (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        address TEXT NOT NULL,
        contact_number TEXT NOT NULL
    )''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Senders (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        address TEXT NOT NULL,
        contact_number TEXT NOT NULL
    )''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Shipments (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        package_id INTEGER NOT NULL,
        sender_id INTEGER NOT NULL,
        receiver_id INTEGER NOT NULL,
        shipping_method TEXT NOT NULL,
        tracking_number TEXT NOT NULL,
        status TEXT NOT NULL,
        FOREIGN KEY (package_id) REFERENCES Packages(id),
        FOREIGN KEY (sender_id) REFERENCES Senders(id),
        FOREIGN KEY (receiver_id) REFERENCES Receivers(id)
    )''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Claims (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        shipment_id INTEGER NOT NULL,
        claim_status TEXT NOT NULL,
        claim_date DATE NOT NULL,
        resolution_date DATE,
        amount_claimed REAL NOT NULL,
        FOREIGN KEY (shipment_id) REFERENCES Shipments(id)
    )''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS CustomerFeedback (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        shipment_id INTEGER NOT NULL,
        customer_id INTEGER NOT NULL,
        rating INTEGER NOT NULL,
        comments TEXT NOT NULL,
        feedback_date DATE NOT NULL,
        FOREIGN KEY (shipment_id) REFERENCES Shipments(id),
        FOREIGN KEY (customer_id) REFERENCES Senders(id)
    )''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS DeliveryAttempts (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        shipment_id INTEGER NOT NULL,
        attempt_date DATE NOT NULL,
        status TEXT NOT NULL,
        notes TEXT NOT NULL,
        FOREIGN KEY (shipment_id) REFERENCES Shipments(id)
    )''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS DeliveryLocations (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        shipment_id INTEGER NOT NULL,
        longitude REAL NOT NULL,
        latitude REAL NOT NULL,
        delivery_date DATE NOT NULL,
        FOREIGN KEY (shipment_id) REFERENCES Shipments(id)
    )''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS ServiceAreas (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        service_area_name TEXT NOT NULL,
        coverage_description TEXT NOT NULL,
        is_active BOOLEAN NOT NULL
    )''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS ShippingRates (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        shipping_method TEXT NOT NULL,
        weight_limit REAL NOT NULL,
        base_price REAL NOT NULL,
        additional_cost_per_kg REAL NOT NULL
    )''')

# Commit the changes to create the tables
create_tables()
conn.commit()

# Helper functions to generate random data

# Custom content descriptions for packages
package_content_descriptions = [
    "Electronics - Fragile", "Clothing - Casual Wear", "Books - Hardcover",
    "Household Items", "Toys - Children", "Sports Equipment", "Jewelry - Delicate",
    "Documents - Confidential", "Groceries - Perishable", "Pharmaceuticals"
]

# Custom notes for delivery attempts
delivery_attempt_notes = [
    "Receiver unavailable at the time of delivery.",
    "Delivery rescheduled for the next day.",
    "Address not found, please update delivery information.",
    "Package left with neighbor as per instructions.",
    "Failed due to weather conditions.",
    "Receiver requested delivery after working hours.",
    "Package handed to building concierge.",
    "Delivery successful, signature obtained.",
    "Unable to deliver, package damaged in transit."
]

# Custom coverage descriptions for service areas
service_area_descriptions = [
    "Covers downtown and neighboring suburbs.",
    "Offers next-day delivery across the city.",
    "Primarily services rural and hard-to-reach areas.",
    "Specializes in industrial and business districts.",
    "Full coverage of residential areas.",
    "Covers coastal cities with express delivery.",
    "Delivery limited to commercial areas only.",
    "Extended coverage to remote villages.",
    "Next-day delivery across all urban zones."
]

# Generate dates from the last 3 years
def generate_recent_date():
    return fake.date_between(start_date="-3y", end_date="today")

# Generate package data
def create_packages(n):
    for _ in range(n):
        cursor.execute('''INSERT INTO Packages (weight, length, width, height, content_description) 
                          VALUES (?, ?, ?, ?, ?)''', 
                          (random.uniform(1, 10), random.uniform(10, 100), random.uniform(10, 100),
                           random.uniform(10, 100), random.choice(package_content_descriptions)))
    conn.commit()
    print(f"Inserted {n} packages.")

# Generate receiver data
def create_receivers(n):
    for _ in range(n):
        cursor.execute('''INSERT INTO Receivers (name, address, contact_number) 
                          VALUES (?, ?, ?)''', 
                          (fake.name(), fake.address(), fake.phone_number()))
    conn.commit()
    print(f"Inserted {n} receivers.")

# Generate sender data
def create_senders(n):
    for _ in range(n):
        cursor.execute('''INSERT INTO Senders (name, address, contact_number) 
                          VALUES (?, ?, ?)''', 
                          (fake.name(), fake.address(), fake.phone_number()))
    conn.commit()
    print(f"Inserted {n} senders.")

# Generate shipment data
def create_shipments(n):
    for _ in range(n):
        cursor.execute('''INSERT INTO Shipments (package_id, sender_id, receiver_id, shipping_method, tracking_number, status) 
                          VALUES (?, ?, ?, ?, ?, ?)''', 
                          (random.randint(1, num_packages), random.randint(1, num_senders), random.randint(1, num_receivers),
                           random.choice(['Air', 'Ground', 'Sea']), fake.uuid4(), random.choice(['Pending', 'Delivered'])))
    conn.commit()
    print(f"Inserted {n} shipments.")

# Generate claim data
def create_claims(n):
    for _ in range(n):
        cursor.execute('''INSERT INTO Claims (shipment_id, claim_status, claim_date, resolution_date, amount_claimed) 
                          VALUES (?, ?, ?, ?, ?)''', 
                          (random.randint(1, num_shipments), random.choice(['Open', 'Resolved']), 
                           generate_recent_date(), generate_recent_date(), random.uniform(10, 1000)))
    conn.commit()
    print(f"Inserted {n} claims.")

# Generate customer feedback data
# Feedback comments updated with corresponding ratings
positive_feedback_comments = [
    ("The package arrived on time and in great condition.", 5),
    ("The packaging was secure, and I appreciate the care taken.", 5),
    ("The delivery person was very polite and professional.", 5),
    ("Overall a smooth experience, would use again!", 4),
    ("Excellent service! Will definitely recommend.", 5),
    ("The delivery was successful, but I would prefer more updates.", 4)
]

neutral_feedback_comments = [
    ("Delivery was delayed, but the service was responsive.", 3),
    ("Had a minor issue with the tracking number, but it was resolved quickly.", 3),
    ("Quick delivery but the tracking updates were lacking.", 3)
]

negative_feedback_comments = [
    ("Received the package, but it was damaged. Needs better handling.", 2),
    ("The delivery was delayed too long and I received no updates.", 1),
    ("Package left in an unsafe location, not happy with the service.", 1),
    ("Address was wrong, leading to a late delivery.", 2),
    ("The delivery driver was rude and unhelpful.", 1),
    ("The package was lost in transit, very disappointing.", 1)
]

# Combine all feedback comments
all_feedback_comments = positive_feedback_comments + neutral_feedback_comments + negative_feedback_comments

# Generate customer feedback with ratings that correspond to reviews
def create_customer_feedback(n):
    for _ in range(n):
        feedback = random.choice(all_feedback_comments)
        cursor.execute('''INSERT INTO CustomerFeedback (shipment_id, customer_id, rating, comments, feedback_date) 
                          VALUES (?, ?, ?, ?, ?)''', 
                          (random.randint(1, num_shipments), random.randint(1, num_senders), 
                           feedback[1], feedback[0], generate_recent_date()))
    conn.commit()
    print(f"Inserted {n} customer feedback entries.")



# Define potential statuses and notes for delivery attempts
attempt_statuses = ["Successful", "Failed", "Attempted"]
attempt_notes = {
    "Successful": [
        "Package delivered successfully.",
        "Receiver unavailable at the time of delivery.",  # Special case for Successful status
        "Left package with a neighbor."
    ],
    "Failed": [
        "Address incorrect, unable to deliver.",
        "Delivery delayed due to weather conditions.",
        "Package returned due to address issue."
    ],
    "Attempted": [
        "Customer unavailable, will attempt again tomorrow."
    ]
}

unique_shipment_ids = list(range(1, 2501))
random.shuffle(unique_shipment_ids)  # Shuffle the IDs for randomness

# Index to track which shipment ID to use next
shipment_id_index_attempts = 0
shipment_id_index_locations = 0

# Function to create delivery attempts with unique shipment IDs
def create_delivery_attempts(n):
    global shipment_id_index_attempts  # Use a unique global index for attempts
    for _ in range(n):
        if shipment_id_index_attempts >= len(unique_shipment_ids):
            print("All unique shipment IDs have been used for attempts.")
            break
        shipment_id = unique_shipment_ids[shipment_id_index_attempts]
        shipment_id_index_attempts += 1

        status = random.choice(attempt_statuses)
        notes = random.choice(attempt_notes[status])
        cursor.execute('''INSERT INTO DeliveryAttempts (shipment_id, attempt_date, status, notes) 
                          VALUES (?, ?, ?, ?)''', 
                          (shipment_id, generate_recent_date(), status, notes))
    conn.commit()
    print(f"Inserted {n} delivery attempt entries.")

def create_delivery_locations(n):
    global shipment_id_index_locations  # Use a unique global index for locations
    for _ in range(n):
        if shipment_id_index_locations >= len(unique_shipment_ids):
            print("All unique shipment IDs have been used for locations.")
            break
        shipment_id = unique_shipment_ids[shipment_id_index_locations]
        shipment_id_index_locations += 1

        cursor.execute('''INSERT INTO DeliveryLocations (shipment_id, longitude, latitude, delivery_date) 
                          VALUES (?, ?, ?, ?)''', 
                          (shipment_id, float(fake.longitude()), float(fake.latitude()), generate_recent_date()))
    conn.commit()
    print(f"Inserted {n} delivery locations.")


# Generate service areas data with realistic coverage descriptions
# Define a mapping for logical `coverage_description` and `is_active` combinations
coverage_description_map = {
    "Fully operational": True,
    "High demand zone": True,
    "Under maintenance": False,
    "Restricted access": False,
    "Limited coverage": False,
}

def create_service_areas(n):
    for _ in range(n):
        # Randomly select a coverage description and its corresponding is_active value
        coverage_description = random.choice(list(coverage_description_map.keys()))
        is_active = coverage_description_map[coverage_description]

        cursor.execute('''INSERT INTO ServiceAreas (service_area_name, coverage_description, is_active) 
                          VALUES (?, ?, ?)''', 
                          (fake.city(), coverage_description, is_active))
    conn.commit()
    print(f"Inserted {n} service areas.")


# Generate shipping rates data
def create_shipping_rates(n):
    for _ in range(n):
        shipping_method = random.choice(['Air', 'Ground', 'Sea'])
        
        # Define realistic ranges for weight limits, base prices, and additional costs
        if shipping_method == 'Air':
            weight_limit = random.uniform(1, 50)  # Lower weight limit
            base_price = random.uniform(50, 150)  # Higher base price
            additional_cost_per_kg = random.uniform(5, 15)  # Higher additional cost
        elif shipping_method == 'Ground':
            weight_limit = random.uniform(20, 100)  # Medium weight limit
            base_price = random.uniform(30, 80)  # Moderate base price
            additional_cost_per_kg = random.uniform(3, 8)  # Moderate additional cost
        elif shipping_method == 'Sea':
            weight_limit = random.uniform(50, 500)  # Higher weight limit
            base_price = random.uniform(20, 60)  # Lower base price
            additional_cost_per_kg = random.uniform(1, 5)  # Lower additional cost
        
        cursor.execute('''INSERT INTO ShippingRates (shipping_method, weight_limit, base_price, additional_cost_per_kg) 
                          VALUES (?, ?, ?, ?)''', 
                          (shipping_method, weight_limit, base_price, additional_cost_per_kg))
    conn.commit()
    print(f"Inserted {n} realistic shipping rates.")


# Set the number of records to create
num_packages = 1500
num_senders = 1200
num_receivers = 1200
num_shipments = 2500
num_delivery_attempts = 2500
num_delivery_locations = 1500
num_shipping_rates = 20
num_service_areas = 50
num_customer_feedback = 2000
num_claims = 500


# Generate all data
create_packages(num_packages)
create_receivers(num_receivers)
create_senders(num_senders)
create_shipments(num_shipments)
create_claims(num_claims)
create_customer_feedback(num_customer_feedback)
create_delivery_attempts(num_delivery_attempts)
create_delivery_locations(num_delivery_locations)
create_service_areas(num_service_areas)
create_shipping_rates(num_shipping_rates)

# Close the database connection
conn.close()


Inserted 1500 packages.
Inserted 1200 receivers.
Inserted 1200 senders.
Inserted 2500 shipments.
Inserted 500 claims.
Inserted 2000 customer feedback entries.
Inserted 2500 delivery attempt entries.
Inserted 1500 delivery locations.
Inserted 50 service areas.
Inserted 20 realistic shipping rates.


In [14]:
import os
import shutil

# Get the path to the desktop
desktop_path = os.path.join(os.path.join(os.environ['USERPROFILE']), 'Desktop')  # For Windows

# For MacOS/Linux, uncomment the line below:
# desktop_path = os.path.join(os.path.join(os.path.expanduser('~')), 'Desktop')

# Path to the SQLite database file
db_file = 'courier_services13.db'

# Define the target path (your desktop)
target_path = os.path.join(desktop_path, 'courier_services13.db')

# Copy the file to your desktop
shutil.copy(db_file, target_path)

print(f"Database has been copied to your desktop: {target_path}")


Database has been copied to your desktop: C:\Users\chait\Desktop\courier_services13.db


In [15]:
import sqlite3

def export_sqlite_to_sql(db_file, sql_file):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    # Open the SQL file for writing
    with open(sql_file, 'w') as f:
        # Dump the schema (tables)
        for line in conn.iterdump():
            f.write('%s\n' % line)

    # Close the connection
    conn.close()

# Usage
db_file = 'courier_services13.db'  # Replace with your SQLite database file
sql_file = 'courier_services13.sql'  # Replace with the desired SQL output file name
export_sqlite_to_sql(db_file, sql_file)

print(f"Exported {db_file} to {sql_file} successfully.")


Exported courier_services13.db to courier_services13.sql successfully.
